In [1]:
import csv
import json
import requests
import sys
import oasis


from pyArango.connection import *
from pyArango.collection import Collection, Edges, Field
from pyArango.graph import Graph, EdgeDefinition
from pyArango.collection import BulkOperation as BulkOperation

In [2]:
print("ITSM Data")
print()
!head -n 3 data/pp_recoded_incident_event_log.csv 


ITSM Data

number,sys_updated_at,reassigned,assignment_group,assigned_to,incident_state,knowledge,active,location,urgency,priority,caused_by,u_symptom,reassignment_count,category,closed_code,resolved_by,opened_by,impact,vendor,subcategory,rfc,problem_id,u_priority_confirmation,contact_type,cmdb_ci,D_sys_mod_count,D_reopen_count
INC0000045,2016-05-03 12:00:00,0,2,1,316,323,326,327,551,554,1,561,1086,1114,1172,1189,1405,1612,1,1619,1,1,2306,2308,1,2365,2368
INC0000047,2016-06-03 10:00:00,1,4,80,316,323,326,328,551,554,1,562,1087,1115,1172,1190,1406,1612,1,1620,1,1,2306,2308,1,2366,2368


In [3]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials()

# Connect to the temp database
conn = oasis.connect_pyarango(login)
db = conn[login["dbName"]] 

Reusing cached credentials.


In [4]:
from pyArango.collection import Collection, Field
from pyArango.graph import Graph, EdgeDefinition


class incident(Collection):
    _fields = {
        "D_sys_mod_count": Field(),
        "D_reopen_count": Field(),
        "urgency": Field(),
        'incident_state': Field(),
        'u_symptom': Field(),
        'impact': Field(),
        'contact_type': Field(),
        'u_priority_confirmation' : Field(),
        'cmdb_ci': Field(),
        'rfc': Field(),
        'problem_id': Field(),
        'caused_by': Field(),
        'location': Field(),
        'knowledge': Field(),
        'resolved_by': Field(),
        'subcategory': Field(),
        'active': Field(),
        'category': Field(),
        'priority': Field(),
        'reassigned': Field()
        
    }
    
class support_org(Collection):
    _fields = {
        "assigned_to": Field(),
        "assignment_group": Field()
    }
class customer(Collection):
    _fields = {
        "opened_by": Field()
    }
class vendor(Collection):
    _fields = {
        "vendor": Field()
    }

class incident_support_org(Edges): 
    _fields = {
        #user_id and item_id are encoded by _from, _to 
    }

class incident_customer(Edges): 
    _fields = {
        #user_id and item_id are encoded by _from, _to 
    }
class incident_vendor(Edges): 
    _fields = {
        #user_id and item_id are encoded by _from, _to 
    }

class ITSMGraph(Graph) :
    _edgeDefinitions = [EdgeDefinition("incident_support_org", \
                                       fromCollections=["incident"], toCollections=["support_org"]),
                        EdgeDefinition("incident_customer", \
                                       fromCollections=["incident"], toCollections=["customer"]),
                        EdgeDefinition("incident_vendor", \
                                       fromCollections=["incident"], toCollections=["vendor"])
                       
                       ]
    _orphanedCollections = []

db.createCollection("incident")
db.createCollection("support_org")
db.createCollection("customer")
db.createCollection("vendor")
db.createCollection("incident_support_org")
db.createCollection("incident_customer")
db.createCollection("incident_vendor")
itsm_graph = db.createGraph("ITSMGraph")

print("Collection/Graph Setup done.")

Collection/Graph Setup done.


In [ ]:
collection = db["Users"]
with BulkOperation(collection, batchSize=100) as col:
    with open('data/users.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        #Skip header
        next(reader)
        for row in reader:
            user_id,age,gender,occupation,zip = tuple(row)
            doc = col.createDocument()
            doc["_key"] = user_id
            doc["age"] = age
            doc["gender"] = gender
            doc.save()

collection = db["Movies"]
with BulkOperation(collection, batchSize=100) as col:
    with open('data/movies.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        #Skip header
        next(reader)
        for row in reader:
            movie_id, movie_title , release_date , video_release_date , url , unknown , action , adventure , animation , childrens , comedy , crime , documentary , drama , fantasy , noir , horror , musical , mystery , romance , scifi , thriller , war , western = tuple(row)
            doc = col.createDocument()
            doc["_key"] = movie_id
            doc["movie_title"] = movie_title
            doc["release_date"] = release_date
            doc.save()

collection = db["Ratings"]
with BulkOperation(collection, batchSize=1000) as col:
    with open('data/ratings.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        #Skip header
        next(reader)
        for row in reader:
            user_id,movie_id,rating,timestamp = tuple(row)
            doc = col.createDocument()
            doc["_from"] = "Users/"+user_id
            doc["_to"] = "Movies/"+movie_id
            doc["ratings"] = rating
            doc["timestamp"] = timestamp
            doc.save()
        
print("Import Done")